In [ ]:
import obspy
qmlfile = '/data/Pinatubo/PHASE/all_events.xml'
cat = obspy.read_events(qmlfile)
print(cat)


In [ ]:
def print_event(ev, times_only=False):
    for origin in ev.origins:
        if times_only:
            print(origin.time)
        else:
            print(origin)    
    for pick in ev.picks:
        if times_only:
            print(pick.time, pick.waveform_id)
        else:
            print(pick)



In [ ]:
print(obspy.UTCDateTime(2000,1,1,3,1,2.532))

In [ ]:
for event in catalog:
    bad_event = False
    otime = event.origins[0].time
    wrong_picks = []
    for pick in event.picks:
        if abs(pick.time - otime)>60:
            wrong_picks.append(pick)
            #event.picks.remove(pick)
    Nbad = len(wrong_picks)
    Ngood = len(event.picks)-Nbad
    if len(wrong_picks)>0:
            print(f'Event {otime}: #good={Ngood}, #bad={Nbad}')
            if 
            sorted_times = sorted([p.time for p in wrong_picks])
            min_time = sorted_times[0]
            for 
            
            '''
            print('****** BAD PICK TIME *********')
            print(pick)
            print_event(event, times_only=True)
            print('Enter new minute and second values', flush=True)
            thismin = input('new minute')
            thissec = input('new seconds')
            pick.time = obspy.UTCDateTime(otime.year, otime.month, otime.day, otime.hour, int(thismin), float(thissec))
            print(f'new pick time={pick.time}')
            '''


In [ ]:
from obspy.core.event import Catalog, Event, Origin, Pick
from obspy.core import UTCDateTime

def separate_bad_picks(catalog, time_threshold=60):
    """
    Identifies picks that are too far (> time_threshold) from their event's origin time,
    removes them from the original events, and creates new Event objects.

    Parameters:
        catalog (obspy.core.event.Catalog): The ObsPy Catalog containing multiple events.
        time_threshold (int, optional): The maximum allowed time difference (default: 60s).

    Returns:
        obspy.core.event.Catalog: A new catalog with fixed events and separate bad pick events.
    """
    new_catalog = Catalog()  # Store fixed events
    bad_picks = []  # Collect picks that need to be in separate events

    for event in catalog:
        if not event.origins:
            new_catalog.append(event)  # Skip if no origin time
            continue
        
        origin_time = event.origins[0].time
        valid_picks = []
        
        # Separate bad picks
        for pick in event.picks:
            if abs((pick.time - origin_time)) > time_threshold:
                bad_picks.append(pick)  # Collect for new event
            else:
                valid_picks.append(pick)  # Keep in original event

        # Update original event with only valid picks
        if valid_picks:
            event.picks = valid_picks
            new_catalog.append(event)  # Save the cleaned event

    # 🔹 Step 2: Create new events from bad picks
    while bad_picks:
        first_pick = bad_picks.pop(0)  # Take the first pick to start a new event
        sorted_times = sorted([p.time for p in bad_picks])
        min_time = sorted_times[0]        
        new_event = Event()
        #new_event.picks.append(first_pick)

        # Find other picks close in time
        related_picks = [p for p in bad_picks if abs(p.time - min_time) < time_threshold]
        for p in related_picks:
            new_event.picks.append(p)
            bad_picks.remove(p)  # Remove from processing

        # Set an estimated origin time based on earliest pick
        estimated_origin = Origin(time=min(p.time for p in new_event.picks))
        new_event.origins.append(estimated_origin)
        new_catalog.append(new_event)

    return new_catalog

# Example usage:
#catalog = Catalog.read("all_events.xml", format="QUAKEML")  # Load the original catalog
fixed_catalog = separate_bad_picks(catalog, time_threshold=60)

# Save the fixed catalog back to QuakeML
fixed_catalog.write("/data/Pinatubo/PHASE/fixed_events.xml", format="QUAKEML")
print("\n✅ Fixed catalog saved as 'fixed_events.xml'")


In [ ]:
import os
#from obspy.core.event import Catalog
from obspy.io.nordic.core import _write_nordic
def save_catalog_to_seisan(catalog, output_dir):
    """
    Saves an ObsPy Catalog as Seisan REA database S-files in Nordic format using `_write_nordic`.

    Parameters:
        catalog (obspy.core.event.Catalog): The ObsPy Catalog containing multiple events.
        output_dir (str): Directory where S-files will be saved.

    Returns:
        None
    """
    

    os.makedirs(output_dir, exist_ok=True)  # Ensure the REA directory exists

    for event in catalog:
        # Get the origin time (used for the filename)
        if not event.origins:
            print(f"Skipping event {event} (no origin time)")
            continue

        
        origin_time = event.origins[0].time  # Use the first origin time
        ymdir = os.path.join(output_dir, f'{origin_time.year:4d}', f'{origin_time.month:02d}')
        if not os.path.isdir(ymdir):
            os.makedirs(ymdir)
        
        # Note: do not actually need to generate filename, as passing None forces ObsPy to generate it
        filename = origin_time.strftime("%d-%H%M-%S") + "L.S" + origin_time.strftime("%Y%m")  # Seisan S-file naming convention
        file_path = os.path.join(ymdir, filename)   
        #print(file_path)   

        # Write the event to a Nordic file using `_write_nordic`
        try:
            _write_nordic(event, filename, userid='gt', evtype='L', outdir=ymdir, wavefiles=None, 
                      explosion=False, nordic_format='OLD', overwrite=True, high_accuracy=False)
        except Exception as e:
            print('\nBAD EVENT: ',file_path)
            print(e)
            print_event(event)
            #input('ENTER')
            continue
        #else:
        #    print(f"✅ Saved Seisan S-file: {file_path}")

# Example usage:
#catalog = obspy.read_events("/data/Pinatubo/PHASE/all_events.xml", format="QUAKEML")  # Load the QuakeML catalog
output_directory = "/data/SEISAN_DB/REA/PINAT"  # Example REA structure: REA/NET/YEAR-MONTH

# Convert and save the Catalog to Seisan
save_catalog_to_seisan(fixed_catalog, output_directory)


In [ ]:
import os
import glob
from obspy import read, UTCDateTime
from obspy.core.event import Catalog, Comment

def associate_miniseed_with_events(catalog, wav_directory):
    """
    Associates MiniSEED files from a Seisan WAV database with Pick objects in an ObsPy Catalog.
    If a Pick's time falls within a MiniSEED file's time range, a Comment is added to the Event.

    Parameters:
        catalog (obspy.core.event.Catalog): The ObsPy Catalog containing Events with Picks.
        wav_directory (str): Path to the Seisan WAV database (e.g., "WAV/").

    Returns:
        obspy.core.event.Catalog: The updated Catalog with associated MiniSEED file paths in Comments.
    """

    # 🔹 Step 1: Load MiniSEED files and extract time windows
    miniseed_files = sorted(glob.glob(os.path.join(wav_directory, "PINAT/1991/??/1991*"), recursive=True))  # Adjust pattern if needed
    miniseed_time_ranges = []

    for file in miniseed_files:
        try:
            stream = read(file)
            start_time = min(tr.stats.starttime for tr in stream)
            end_time = max(tr.stats.endtime for tr in stream)
            miniseed_time_ranges.append((file, start_time, end_time))
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")
    
    print(f"✅ Loaded {len(miniseed_time_ranges)} MiniSEED files for association.")

    # 🔹 Step 2: Iterate through Events & Picks to check time association
    for event in catalog:
        associated_files = set()  # Track unique MiniSEED files per event
        
        for pick in event.picks:
            pick_time = pick.time  # Extract pick time

            # Check if Pick falls within any MiniSEED file time range
            for file_path, start_time, end_time in miniseed_time_ranges:
                if start_time <= pick_time <= end_time:
                    associated_files.add(file_path)

        # 🔹 Step 3: Add MiniSEED file paths as Comments in Event
        if associated_files:
            comment_text = "Associated MiniSEED files:\n" + "\n".join(sorted(associated_files))
            event.comments.append(Comment(text=comment_text))
            print(f"📌 Added MiniSEED association for Event at {event.origins[0].time}")

    return catalog

# Example usage:
#catalog = Catalog.read("fixed_events.xml", format="QUAKEML")  # Load corrected catalog
wav_directory = "/data/SEISAN_DB/WAV/"  # Path to Seisan WAV database

# Associate MiniSEED files with events
updated_catalog = associate_miniseed_with_events(fixed_catalog, wav_directory)

# Save the updated catalog with comments
updated_catalog.write("/data/Pinatubo/PHASE/events_with_miniseed.xml", format="QUAKEML")
print("\n✅ Updated catalog saved as 'events_with_miniseed.xml'")
